In [1]:
import pandas as pd

In [7]:
filepath = "/Users/colinyao/Desktop/Code/FINM/Portfolio_36700/HW/PortfolioRiskHW/HW3/data/spx_returns_weekly.xlsx"
ret_df = pd.read_excel(filepath, sheet_name=2, index_col=0)

In [9]:
instruments = ['AAPL', 'META', 'NVDA', 'TSLA']
sel_df = ret_df[instruments]

In [27]:
# 1.1
def compstat(series, sname):
    variance = series.var()
    empvar = series.quantile(0.05)
    empcvar = series[series <= empvar].mean()
    print(f"Instrument {sname}: Variance {variance}, empirical VaR {empvar}, empirical CVaR {empcvar}")
    return (sname, variance, empvar, empcvar)

csdict = {}
for i in instruments:
    csdict[i] = compstat(sel_df[i], i)
pass

Instrument AAPL: Variance 0.0014716081925336146, empirical VaR -0.056366297517849974, empirical CVaR -0.08312491891904249
Instrument META: Variance 0.0023737852999561686, empirical VaR -0.07001167040022517, empirical CVaR -0.1031960640416827
Instrument NVDA: Variance 0.004127520553944743, empirical VaR -0.08685282018109243, empirical CVaR -0.11645514176034043
Instrument TSLA: Variance 0.006613492943319799, empirical VaR -0.11739730351568925, empirical CVaR -0.14781370062338312


In [24]:
# 1.2
equiweights = pd.Series(sum((1 / len(instruments)) * sel_df[i] for i in instruments), name="equal_weights")
ew_stats = compstat(equiweights, equiweights.name)
pass

Instrument equal_weights: Variance 0.0019147546766932891, empirical VaR -0.06194990495722183, empirical CVaR -0.08499232001174674


# 1.2
We note that the equal-weights portfolio has variance close to the minimum variance of the component instruments. Similarly, empirical VaR and empirical CVaR are close to the minimum empirical VaR and empirical CVaR of the component instruments. This is because the instruments are somewhat diversified, so the variance of their weighted sum is lesser compared to the equivalent sum of their variances, and similarly for VaR and CVaR. 

In [29]:
# 1.3
most_volatile = sorted(((i, sel_df[i].var()) for i in instruments), key=lambda x: x[1], reverse=True)[0][0] # most volatile instrument
instr_dropped = [i for i in instruments if i != most_volatile]
equidropweights = pd.Series(sum((1 / len(instruments)) * sel_df[i] for i in instr_dropped), name="equal_dropped_weights")
ewd_stats = compstat(equidropweights, equidropweights.name)

# comparison
print(f"Difference due to dropping: Variance {ew_stats[1] - ewd_stats[1]}, empirical VaR {ew_stats[2] - ewd_stats[2]}, empirical CVaR {ew_stats[3] - ewd_stats[3]}")

print(f"Stats of the most variant (scaled): Variance {(1 / len(instruments)) * csdict[most_volatile][1]}, empirical VaR {(1 / len(instruments)) * csdict[most_volatile][2]}, empirical CVaR {(1 / len(instruments)) * csdict[most_volatile][3]}")

Instrument equal_dropped_weights: Variance 0.0009169845136893209, empirical VaR -0.042462511580251314, empirical CVaR -0.060531130154127574
Difference due to dropping: Variance 0.0009977701630039682, empirical VaR -0.019487393376970516, empirical CVaR -0.024461189857619164
Stats of the most variant (scaled): Variance 0.0016533732358299498, empirical VaR -0.029349325878922312, empirical CVaR -0.03695342515584578


# 1.3
The risk which the asset adds to the portfolio is as stated in "Difference due to dropping" above. Note that each of the relevant quantities is lesser in magnitude compared to the (scaled) solo asset: this is because some of the variance in the new asset (were it added to the portfolio) is diversified away by the other assets. Similarly, losses as reflected in VaR and CVaR are counterbalanced in some cases by gains in the other assets. 